In [1]:
%load_ext autoreload
%autoreload 2

In [449]:
import matplotlib as mpl
from matplotlib import rc
rc('text',usetex=False)
rc('font',**{'family':'serif','serif':['Times New Roman'],'size':8})
rc('mathtext',fontset='cm')
import matplotlib.pyplot as plt
% matplotlib inline
% config InlineBackend.figure_format = 'svg'
% config InlineBackend.rc = {'figure.figsize': (5,3.5)}

In [3]:
import numpy as np
from numpy import sqrt,log,array as ar
import numpy.linalg as la
import cvxpy as cvx
from helper import *
from cd.model.utility import LipschitzExpUtility as leu, RiskNeutralUtility as rnu
from cd.model.distrs import RademacherDistribution,UniformDistribution
import cd.model.synth_data as synth
import scaler
from itertools import product

In [4]:
p = 100
R = RademacherDistribution()
Xs = [RademacherDistribution() for _ in range(p)]

In [5]:
M = synth.GaussianMarket(Xs,R)
t_true = M.sample_t(5_000)
u = leu(1)

In [6]:
la.norm(t_true.mean(axis=0))**2

0.41772496000000003

In [237]:
xticks = np.arange(10,51,5)
ns = np.arange(9,51)
pws = np.array([1/2,3/4,1])
# ps = np.array(2.5*ns**pws[:,None],dtype='int')
ps1 = np.array(2*ns,dtype='int')
ps34 = np.array(3.5*ns**(3/4),dtype='int')
ps12 = np.array(6*sqrt(ns),dtype='int')
ps = np.array([ps12,ps34,ps1])

ps32 = np.array(0.68*ns**(3/2),dtype='int')
ns32 = ns[ps32<=100]
ps32 = ps32[ps32<=100]


In [238]:
ps32,ns32

(array([ 18,  21,  24,  28,  31,  35,  39,  43,  47,  51,  56,  60,  65,
         70,  75,  79,  85,  90,  95, 100]),
 array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
        26, 27, 28]))

In [239]:
nns = np.linspace(9,50,100)
fig,ax = plt.subplots(1)
ax.plot(nns,2*nns,'C0-',label=ords % '')
ax.plot(nns,3.5*nns**(3/4),'C0-s',ms=4,markevery=6,label=ords % '^{3/4}')
ax.plot(nns,6*sqrt(nns),'C0-o',ms=4,markevery=6,label=ords % '^{1/2}')
ax.set_xlabel('$n$')
ax.set_ylabel('$p$')
ax.set_xticks(xticks)
ax.set_yticks(np.arange(20,101,10))
ax.axis(ymax=100)
ax.legend()
# plt.savefig('fig/np_np.pdf',bbox_inches='tight',pad_inches=0)



## Sous optimalité

In [133]:
p = 100
R = RademacherDistribution()
Xs = [RademacherDistribution() for _ in range(p)]

M = synth.GaussianMarket(Xs,R,corr_vector=[0 for _ in range(p)])
t_true = M.sample_t(1_000)

In [135]:
m = 50
erroru = np.zeros((len(ns),len(ps)))
errorce = np.zeros((len(ns),len(ps)))
boundu = np.zeros((len(ns),len(ps)))
boundce = np.zeros((len(ns),len(ps)))
ts = create_sample(t_true,m,max(ns))
for i,n in enumerate(ns):
    if i%5 == 0: print(i)
    for j,p in enumerate(ps):
        p = p[i]
        t = ts[:,:n,p]
        q_true = solve(t_true[:,:p],u=u,lamb=0)[0]
        qs = solve(t,u,lamb=1)
        erroru[i,j] = out_error(qs,u,t_true,lamb=1).min()
        errorce[i,j] = u.inverse(erroru[i,j])
        boundu[i,j] = bound_sou(n,p,q_true,t_true[:,:p],u)
        boundce[i,j] = bound_soce(n,p,q_true,t_true[:,:p],u,erroru[i,j])


0


5


10


15


20


25


In [136]:
plt.plot(erroru)

In [131]:
plt.plot(boundu)

In [105]:
ns,ps

(array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
        26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,
        43, 44, 45, 46, 47, 48, 49, 50]),
 array([[ 18,  18,  19,  20,  21,  22,  23,  24,  24,  25,  26,  26,  27,
          28,  28,  29,  30,  30,  31,  31,  32,  32,  33,  33,  34,  34,
          35,  36,  36,  36,  37,  37,  38,  38,  39,  39,  40,  40,  41,
          41,  42,  42],
        [ 18,  19,  21,  22,  23,  25,  26,  28,  29,  30,  31,  33,  34,
          35,  36,  37,  39,  40,  41,  42,  43,  44,  45,  47,  48,  49,
          50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,
          63,  64,  65],
        [ 18,  20,  22,  24,  26,  28,  30,  32,  34,  36,  38,  40,  42,
          44,  46,  48,  50,  52,  54,  56,  58,  60,  62,  64,  66,  68,
          70,  72,  74,  76,  78,  80,  82,  84,  86,  88,  90,  92,  94,
          96,  98, 100]]))

In [111]:
ps[:,5]

array([22, 25, 28])

In [107]:
ts = create_sample(t_true,n=max(ns),m=m)

In [117]:
k = 3
error = np.zeros((len(ns),k))
bound = np.zeros((len(ns),k))
errorce = np.zeros((len(ns),k))
boundce = np.zeros((len(ns),k))
for i,n in enumerate(ns):
    if i%5 == 0: print(i)
    for j,p in enumerate(ps[:,i]):
        t = ts[:,:n,:p]
        qs = solve(t,u,lamb=1)
        inerror = in_error(qs,u,t,lamb=1)
        outerror = out_error(qs,u,t_true,lamb=1)
        error[i,j] = np.percentile(inerror - outerror,95)
        errorce[i,j] = np.percentile(u.inverse(inerror) - u.inverse(outerror),95)
        bound[i,j] = bound_genu(n,sqrt(p),lamb=1)
        boundce[i,j] = bound[i,j]*u.subinverse(errorce[i,j])

0


5


10


15


20


25


30


35


40


In [202]:
fig,ax = plt.subplots(2,sharex=True,figsize=(6,8))
pltu,pltce = ax
pltu.plot(ns,error[:,2],'C0-',ms=4,markevery=3,label='$k=1')
pltu.plot(ns,error[:,0],'C0-o',ms=4,markevery=3,label='$k=1/2$')
pltu.plot(ns,error[:,1],'C0-s',ms=4,markevery=3,label='$k=3/4$')
pltu.axis(ymax=1.0)
pltu.legend(loc='lower left')
pltu.set_yticks([0.4,0.5,0.6,0.7])
pltu.tick_params('y',colors='C0')
pltu.set_title("a) Utilit\\'e")

pltu2 = pltu.twinx()
pltu2.plot(ns,bound[:,2],'k--',ms=4,markevery=3,label='$k=1')
pltu2.plot(ns,bound[:,0],'k--o',ms=4,markevery=3,label='$k=1/2$')
pltu2.plot(ns,bound[:,1],'k--s',ms=4,markevery=3,label='$k=3/4$')
pltu2.axis(ymin=-17)
pltu2.set_yticks([25,35,45,55,65,75])
pltu2.legend(loc='upper left')

pltce.plot(ns,errorce[:,2],'C0-',ms=4,markevery=3,label='$k=1')
pltce.plot(ns,errorce[:,0],'C0-o',ms=4,markevery=3,label='$k=1/2$')
pltce.plot(ns,errorce[:,1],'C0-s',ms=4,markevery=3,label='$k=3/4$')
pltce.axis(ymax=1.65)
pltce.legend(loc='lower left')
pltce.set_yticks([0.6,0.8,1.0])
pltce.tick_params('y',colors='C0')

pltce2 = pltce.twinx()
pltce2.plot(ns,bound[:,2],'k--',ms=4,markevery=3,label='$k=1')
pltce2.plot(ns,bound[:,0],'k--o',ms=4,markevery=3,label='$k=1/2$')
pltce2.plot(ns,bound[:,1],'k--s',ms=4,markevery=3,label='$k=3/4$')
pltce2.axis(ymin=-17)
pltce2.set_yticks([25,35,45,55,65,75])
pltce2.legend(loc='upper left')
pltce.set_xlabel('$n$')

# plt.savefig('fig/np_genbound.pdf',bbox_inches='tight',pad_inches=0)


IndexError: too many indices for array

IndexError: too many indices for array

In [210]:
np.arange(ps.min(),ps.max()+1)

array([ 18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100])

In [211]:
q_trues = [solve(t_true[:,:p],u=u,lamb=0)[0] for p in np.arange(ps.min(),ps.max()+1)]

Combined backtracking failed 0 0 90 0 sigma 1


Combined backtracking failed 0 0 90 0 sigma 1


Combined backtracking failed 0 0 90 0 sigma 1


In [215]:
dq = {p:q_true for p,q_true in zip(np.arange(ps.min(),ps.max()+1),q_trues)}

In [219]:
dq[18]

array([ 0.08533925,  0.06164977,  0.08568888,  0.14595338,  0.00254219,
        0.1933462 ,  0.16468629,  0.25845887,  0.16965063,  0.1065275 ,
        0.30414183,  0.23273194,  0.14656326,  0.17192155,  0.16831207,
        0.12272717,  0.08155678,  0.13667088])

In [223]:
eu_trues = [u(t_true[:,:p]@q_true).mean() for p,q_tr2ue in dq.items()]

In [225]:
deus = {p:eu_true for p,eu_true in zip(np.arange(ps.min(),ps.max()+1),eu_trues)}

In [317]:
plt.plot(np.arange(ps.min(),ps.max()+1),[la.norm(q)**2 for p,q in dq.items()])
plt.xlabel('$p$')
plt.savefig('fig/np_normqp.pdf',bbox_inches='tight',pad_inches=0)

In [234]:
ts = create_sample(t_true,n=max(ns),m=50)

In [280]:
k = 3
error = np.zeros((len(ns),k))
bound = np.zeros((len(ns),k))
errorce = np.zeros((len(ns),k))
boundce = np.zeros((len(ns),k))
for i,n in enumerate(ns):
    if i%5 == 0: print(i)
    for j,p in enumerate(ps[:,i]):
        t = ts[:,:n,:p]
        qs = solve(t,u,lamb=1)
        outerror = out_error(qs,u,t_true,lamb=1)
        error[i,j] = np.percentile(deus[p] - outerror,95)
        errorce[i,j] = np.percentile(u.inverse(deus[p]) - u.inverse(outerror),95)
        bound[i,j] = bound_sou(n,sqrt(p),dq[p])
        boundce[i,j] = bound[i,j] * u.subinverse(u.inverse(deus[p]))
        

0


5


10


15


20


25


30


35


40


In [27]:
ns = np.arange(4000,10001)
pws = np.array([1/2,3/4,1])
# ps = np.array(2.5*ns**pws[:,None],dtype='int')
ps1 = np.array(2*ns,dtype='int')
ps34 = np.array(3.5*ns**(3/4),dtype='int')
ps12 = np.array(6*sqrt(ns),dtype='int')
ps = np.array([ps12,ps34,ps1])
# 

In [28]:
k = 3

bound = np.zeros((len(ns),k))
for i,n in enumerate(ns):
    for j,p in enumerate(ps[:,i]):
        bound[i,j] = bound_sou(n,sqrt(p),0)


In [30]:
plt.plot(ns,bound[:,1])

In [304]:
fig,ax = plt.subplots(2,sharex=True,figsize=(6,10))
pltu,pltce = ax
pltu.plot(ns,error[:,2],'C0-',ms=4,markevery=3,label='$k=1$')
pltu.plot(ns,error[:,1],'C0-s',ms=4,markevery=3,label='$k=3/4$')
pltu.plot(ns,error[:,0],'C0-o',ms=4,markevery=3,label='$k=1/2$')
pltu.legend(loc='center left')
pltu.axis(ymax=0.74)
pltu.tick_params('y',colors='C0')
pltu.set_yticks([0.2,0.3,0.4,0.5])
pltu2 = pltu.twinx()
pltu2.plot(ns,bound[:,2],'k--',ms=4,markevery=3,label='$k=1$')
pltu2.plot(ns,bound[:,1],'k--s',ms=4,markevery=3,label='$k=3/4$')
pltu2.plot(ns,bound[:,0],'k--o',ms=4,markevery=3,label='$k=1/2$')
pltu2.axis(ymin=-175)
pltu2.set_yticks([600,800,1000,1200,1400])
pltu.set_title("Utilit\\'e")

pltce.plot(ns,errorce[:,2],'C0-',ms=4,markevery=3,label='$k=1$')
pltce.plot(ns,errorce[:,1],'C0-s',ms=4,markevery=3,label='$k=3/4$')
pltce.plot(ns,errorce[:,0],'C0-o',ms=4,markevery=3,label='$k=1/2$')
pltce.set_xlabel('$n$')
pltce.axis(ymax=1)
pltce.tick_params('y',colors='C0')
pltce.set_yticks([0.2,0.3,0.4,0.5,0.6])

pltce2 = pltce.twinx()
pltce2.plot(ns,boundce[:,2],'k--',ms=4,markevery=3,label='$k=1$')
pltce2.plot(ns,boundce[:,1],'k--s',ms=4,markevery=3,label='$k=3/4$')
pltce2.plot(ns,boundce[:,0],'k--o',ms=4,markevery=3,label='$k=1/2$')
pltce2.axis(ymin=-1600)
pltce2.set_yticks([500,1000,1500,2000,2500])
pltce.set_title("\\'Equivalent certain")
# 
# plt.savefig('fig/np_sobound.pdf',bbox_inches='tight',pad_inches=0)


In [177]:
ns=ns[2:]

In [178]:
ps = np.array(0.0016*ns**3/2,dtype='int')

In [172]:
plt.plot(ns[2:],ps[2:])

In [193]:
ts = create_sample(t_true,n=max(ns),m=100)

In [194]:
error = np.zeros(len(ns))
bound = np.zeros(len(ns))
for i,n in enumerate(ns):
    if i%5 == 0: print(i)
    p = ps[i]
    t = ts[:,:n,:p]
    qs = solve(t,u,lamb=1)
    inerror = in_error(qs,u,t,lamb=1)
    outerror = out_error(qs,u,t,lamb=1)
    error[i] = np.percentile(inerror - outerror,95)
    bound[i] = bound_genu(n,sqrt(p),lamb=1)

0


5


10


15


20


25


30


35


In [206]:
fig,pltu = plt.subplots(1,figsize=(6,4))
pltu.plot(ns[0],error[0],'k--',label='Borne')
pltu.plot(ns,error,'C0',label='$k=3/2$')
pltu.axis(ymax=1)
pltu.legend(loc='upper left')
pltu.tick_params('y',colors='C0')
pltu.set_yticks([0.2,0.3,0.4,0.5])
pltu2 = pltu.twinx()
pltu2.plot(ns,bound,'k--')
pltu2.axis(ymin=-70)
pltu2.set_yticks([0,20,40,60])
pltu.set_xlabel('$n$')

# plt.savefig('fig/np_gen32.pdf',bbox_inches='tight',pad_inches=0)


## Erreur de généralisation

In [148]:
m = 150
erroru = np.zeros((len(ns),len(ps)))
boundu = np.zeros((len(ns),len(ps)))
errorce = np.zeros((len(ns),len(ps)))
boundce = np.zeros((len(ns),len(ps)))
ts = create_sample(t_true,m,max(ns))
for i,n in enumerate(ns):
    if i%5 == 0: print(i)
    for j,p in enumerate(ps):
        t = ts[:,:n,:p[i]]
        qs = solve(t,u,lamb=1)
        inerror = in_error(qs,u,t,lamb=1)
        outerror = out_error(qs,u,t_true,lamb=1)
        erroru[i,j] = error_u(inerror,outerror)
        errorce[i,j] = error_ce(inerror,outerror,u)
        boundu[i,j] = bound_genu(n,sqrt(p[i]),lamb=1)
        boundce[i,j] = bound_gence(n,sqrt(p[i]),u,inerror,outerror,lamb=1) 

40


35


30


25


20


15


10


5


0


In [211]:
fig,ax = plt.subplots(2,figsize=(6,9))
fig.subplots_adjust(hspace=.2)
pltu,pltce = ax
ords = '$p=\mathcal{O}(n%s)$'
borns = 'Borne théorique $\delta = 5\%$'

# pltu.plot(ns[0],erroru[0,1],'',alpha=0,label='Bornes théoriques ($\delta = 5\%$)')
# pltu.plot(ns[0],erroru[0,1],'k--',ms=4,markevery=3,label=(ords % ''))
# pltu.plot(ns[0],erroru[0,1],'k--s',ms=4,markevery=3,label=(ords % '^{3/4}'))
# pltu.plot(ns[0],erroru[0,1],'k--o',ms=4,markevery=3,label=(ords % '^{1/2}'))

pltu.plot(ns[0],erroru[0,1],'',alpha=0,label='95e percentile d\'erreur de\ngénéralisation empirique')
pltu.plot(ns,erroru[:,2],'C0',label=(ords % ''))
pltu.plot(ns,erroru[:,1],'C0-s',ms=4,markevery=3,label=(ords % '^{3/4}'))
pltu.plot(ns,erroru[:,0],'C0-o',ms=4,markevery=3,label=(ords % '^{1/2}'))

pltu.legend(loc='lower left',ncol=1)
pltu.set_xlabel('$n$')
pltu.axis(ymax=1,ymin=0.32)
pltu.tick_params('y',colors='C0')
pltu.set_yticks(np.arange(0.35,0.66,0.05))
pltu.set_xticks(xticks)
pltu.set_title('a) Utilité')
pltu.set_ylabel('Erreur de généralisation (util)')

pltu = pltu.twinx()
pltu.plot(ns[0],boundu[0,1],'',alpha=0,label='Bornes théoriques ($\delta = 5\%$)')
pltu.plot(ns,boundu[:,2],'k--',label=(ords % ''))
pltu.plot(ns,boundu[:,1],'k--s',ms=4,markevery=3,label=(ords % '^{3/4}'))
pltu.plot(ns,boundu[:,0],'k--o',ms=4,markevery=3,label=(ords % '^{1/2}'))
pltu.axis(ymin=2*boundu.min()-boundu.max()-10);
pltu.set_yticks(np.arange(30,71,10))
pltu.legend(loc='upper left',ncol=1)

# pltce.plot(ns[0],errorce[0,1],'k--',ms=4,markevery=3,label=borns)
# pltce.plot(ns[0],errorce[0,1],'k--s',ms=4,markevery=3,label=borns)
# pltce.plot(ns[0],errorce[0,1],'k--o',ms=4,markevery=3,label=borns)
pltce.plot(ns[0],errorce[0,1],'',alpha=0,label='95e percentile d\'erreur de\ngénéralisation empirique')
pltce.plot(ns,errorce[:,2],'C0',label=(ords % ''))
pltce.plot(ns,errorce[:,1],'C0-s',ms=4,markevery=3,label=(ords % '^{3/4}'))
pltce.plot(ns,errorce[:,0],'C0-o',ms=4,markevery=3,label=(ords % '^{1/2}'))
pltce.legend(loc='lower left')
pltce.set_xlabel('$n$')
pltce.axis(ymax=1.6,ymin=0.40)
pltce.tick_params('y',colors='C0')
pltce.set_yticks(np.arange(0.5,1.01,0.1))
pltce.set_xticks(xticks)
pltce.set_title('a) Équivalent certain')
pltce.set_ylabel('Errecer de généralisation (rendement)')

pltce = pltce.twinx()
pltce.plot(ns[0],boundce[0,1],'',alpha=0,label='Bornes théoriques ($\delta = 5\%$)')
pltce.plot(ns,boundce[:,2],'k--',label=(ords % ''))
pltce.plot(ns,boundce[:,1],'k--s',ms=4,markevery=3,label=(ords % '^{3/4}'))
pltce.plot(ns,boundce[:,0],'k--o',ms=4,markevery=3,label=(ords % '^{1/2}'))
pltce.axis(ymin=-45)
pltce.set_yticks(np.arange(50,141,10));
pltce.legend(loc='upper left')

plt.savefig('fig/bound_npgenu.pdf',bbox_inches='tight',pad_inches=0)


## Erreur de généralisation (Haut régime)

In [280]:
ns = np.arange(9,51)
ps1 = np.array(2*ns,dtype='int')
ps34 = np.array(3.5*ns**(3/4),dtype='int')
ps12 = np.array(6*sqrt(ns),dtype='int')
ps = np.array([ps12,ps34,ps1])

ns32 = ns[2:]
ps32 = np.array(0.0016*ns32**3/2,dtype='int')

In [282]:
plt.plot(ns32,ps32)

In [275]:
m = 250
erroru = np.zeros(len(ns32))
boundu = np.zeros(len(ns32))
errorce = np.zeros(len(ns32))
boundce = np.zeros(len(ns32))
ts = create_sample(t_true,m,max(ns32))
for i,n in enumerate(ns32):
    p = ps32[i]
    if i%5 == 0: print(i)
    t = ts[:,:n,:p]
    qs = solve(t,u,lamb=1)
    inerror = in_error(qs,u,t,lamb=1)
    outerror = out_error(qs,u,t_true,lamb=1)
    erroru[i] = error_u(inerror,outerror)
    errorce[i] = error_ce(inerror,outerror,u)
    boundu[i] = bound_genu(n,sqrt(p),lamb=1)
    boundce[i] = bound_gence(n,sqrt(p),u,inerror,outerror,lamb=1) 

35


30


25


20


15


10


5


0


In [344]:
fig,ax = plt.subplots(2,figsize=(6,8))
pltu,pltce = ax
fig.subplots_adjust(hspace=.25)
fig.suptitle('Erreur de généralisation: Régime $p=\mathcal{O}(n^{3/2})$',y=0.93,fontsize=10)
pltu.plot(ns32[0],erroru[0],'k--',label='Borne théorique ($\delta = 5\%)$')
pltu.plot(ns32,erroru,'C0',label='95e percentile d\'erreur empirique')
pltu.set_xlabel('$n$')
pltu.set_ylabel('Util')
pltu.axis(ymax=0.95)
pltu.set_yticks(np.arange(0.2,0.56,0.05))
pltu.tick_params('y',colors='C0')
pltu.set_title('a) Erreur en util')
pltu.legend(loc='upper left')

pltu = pltu.twinx()
pltu.plot(ns32,boundu,'k--')
pltu.axis(ymin=-80)
pltu.set_yticks(np.arange(0,81,10));

pltce.plot(ns32[0],errorce[0],'k--',label='Borne théorique ($\delta = 5\%)$')
pltce.plot(ns32,errorce,'C0',label='95e percentile d\'erreur empirique')
pltce.set_xlabel('$n$')
pltce.set_ylabel('Rendement')
pltce.axis(ymax=1.6)
pltce.set_yticks(np.arange(0.2,0.91,0.1))
pltce.tick_params('y',colors='C0')
pltce.set_title('a) Erreur en rendement')
pltce.legend(loc='upper left')

pltce = pltce.twinx()
pltce.plot(ns32,boundce,'k--')
pltce.axis(ymin=-150)
pltce.set_yticks(np.arange(0,151,25))

plt.savefig('fig/bound_np_np32.pdf',bbox_inches='tight',pad_inches=0)
#


## Erreur de sous optimalité

In [347]:
rangep = np.arange(ps.min(),ps.max()+1)
rangep

array([ 18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100])

In [348]:
q_trues = [solve(t_true[:,:p],u=u,lamb=0)[0] for p in rangep]

Combined backtracking failed 0 0 90 0 sigma 1


In [349]:
dq = dict(zip(rangep,q_trues))

In [350]:
eu_trues = [u(t_true[:,:p]@q_true).mean() for p,q_true in dq.items()]
deus = dict(zip(rangep,eu_trues))

In [353]:
m = 150
error = np.zeros((len(ns),len(ps)))
errorce = np.zeros((len(ns),len(ps)))
bound = np.zeros((len(ns),len(ps)))
boundce = np.zeros((len(ns),len(ps)))
ts = create_sample(t_true,m,max(ns))
for i,n in enumerate(ns):
    if i%5 == 0: print(i)
    for j,p in enumerate(ps[:,i]):
        t = ts[:,:n,:p]
        qs = solve(t,u,lamb=1)
        outerror = out_error(qs,u,t_true,lamb=1)
        error[i,j] = np.percentile(deus[p] - outerror,95)
        errorce[i,j] = np.percentile(u.inverse(deus[p]) - u.inverse(outerror),95)
        bound[i,j] = bound_sou(n,sqrt(p),dq[p],lamb=1)
        boundce[i,j] = bound[i,j] * u.subinverse(u.inverse(deus[p]))


40


35


30


25


20


15


10


5


0


In [456]:
fig,ax = plt.subplots(2,figsize=(6,8))
fig.subplots_adjust(hspace=.25)
pltu,pltce = ax
fig.suptitle('Erreur de sous optimalité — Régimes $p=\mathcal{O}(n^{1/2}),\mathcal{O}(n^{3/4}),\mathcal{O}(n)$',y=0.935,fontsize=10,fontweight='bold')

pltu.plot(ns[0],error[0,1],'',alpha=0,label='Bornes théoriques ($\delta = 5\%$)')
pltu.plot(ns[0],error[0,1],'k--o',ms=4,markevery=4,label=(ords % ''))
pltu.plot(ns[0],error[0,1],'k--s',ms=4,markevery=4,label=(ords % '^{3/4}'))
pltu.plot(ns[0],error[0,1],'k--',ms=4,markevery=4,label=(ords % '^{1/2}'))
pltu.plot(ns[0],error[0,1],'',alpha=0,label='95e percentile d\'erreur de généralisation empirique')
pltu.plot(ns,error[:,2],'C0-o',ms=4,markevery=4,label=(ords % ''))
pltu.plot(ns,error[:,1],'C0-s',ms=4,markevery=4,label=(ords % '^{3/4}'))
pltu.plot(ns,error[:,0],'C0',ms=4,markevery=4,label=(ords % '^{1/2}'))
pltu.legend(loc='lower left',
            bbox_to_anchor = (0.14,-0.035,0,0),
            bbox_transform = plt.gcf().transFigure,
            ncol=2)

pltu.set_xticks(xticks)
pltu.set_xlabel('$n$')
pltu.set_title('a) Utilité')
pltu.set_ylabel('Util')
pltu.axis(ymax=0.6)
pltu.set_yticks(np.arange(0.15,0.41,0.05))
pltu.tick_params('y',colors='C0')

pltu = pltu.twinx()
pltu.plot(ns,bound[:,0],'k--',label=(ords % ''))
pltu.plot(ns,bound[:,1],'k--s',ms=4,markevery=4,label=(ords % '^{3/4}'))
pltu.plot(ns,bound[:,2],'k--o',ms=4,markevery=4,label=(ords % '^{1/2}'))
pltu.axis(ymin=-400)
pltu.set_yticks(np.arange(600,1401,100))


pltce.plot(ns,errorce[:,2],'C0-o',ms=4,markevery=4,label=(ords % ''))
pltce.plot(ns,errorce[:,1],'C0-s',ms=4,markevery=4,label=(ords % '^{3/4}'))
pltce.plot(ns,errorce[:,0],'C0',ms=4,markevery=4,label=(ords % '^{1/2}'))

pltce.set_xticks(xticks)
pltce.set_xlabel('$n$')
pltce.set_title('b) Équivalent certain')
pltce.set_ylabel('Rendement')
pltce.axis(ymax=0.8)
pltce.set_yticks(np.arange(0.1,0.41,0.05))
pltce.tick_params('y',colors='C0')

pltce = pltce.twinx()
pltce.plot(ns,boundce[:,0],'k--')
pltce.plot(ns,boundce[:,1],'k--s',ms=4,markevery=4)
pltce.plot(ns,boundce[:,2],'k--o',ms=4,markevery=4)
pltce.axis(ymin=-1400)
pltce.set_yticks(np.arange(600,2301,200));

plt.savefig('fig/bound_test.pdf',bbox_inches='tight',pad_inches=0)
#